## Time series estimation - Contemporary, daily at system level

Notebook to compare results from Python against R. <br>
%TODO: Check if R-squared are correctly computed.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import plotly.plotly
import plotly.tools as tls
import plotly.graph_objs as go

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
%matplotlib inline
import numpy as np
import datetime as dt
import time

In [3]:
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
info_dict = {'N': lambda x: x.nobs,
             'R2_adj': lambda x: x.rsquared_adj,
             'AIC': lambda x: x.aic,
             'F': lambda x: x.fvalue, 
             'P_F': lambda x: x.f_pvalue, 
             'DW': lambda x: sm.stats.stattools.durbin_watson(x.resid)}

/usr/local/lib/python3.5/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [4]:
from Utils import TransantiagoConstants

In [5]:
DTPMDir = TransantiagoConstants.DTPMDir
DTPM_TRXDir = TransantiagoConstants.DTPM_TRXDir

In [6]:
daily_input_path = os.path.join(DTPM_TRXDir,'3_DAILY/daily_summary.csv')
daily_trx = pd.read_csv(daily_input_path,sep=';',encoding='latin-1', index_col=0)

* Creating new dependent and independent variables

In [7]:
daily_trx.loc[:,'TOTAL_trx'] = daily_trx.loc[:,'pn_SUM_TRX_no_t'] + daily_trx.loc[:,'pn_SUM_TRX_3t'] + daily_trx.loc[:,'pn_SUM_TRX_tm'] + daily_trx.loc[:,'zp_SUM_TRX']

In [8]:
independent_variables_path = os.path.join(DTPM_TRXDir,'0_INDEPENDENTS/independents_variables.csv')
independent_variables = pd.read_csv(independent_variables_path,sep=';',encoding='latin-1', index_col=0, parse_dates=[1])

In [9]:
independent_variables.loc[:,'Verano'] =  independent_variables.loc[:,'Enero'] + independent_variables.loc[:,'Febrero']
independent_variables.loc[:,'Nov_Dic_2017'] = independent_variables.loc[:,'Nov_2017'] + independent_variables.loc[:,'Dic_2017']
independent_variables.loc[:,'WEEK_OF_YEAR'] = independent_variables.loc[:,'DATE'].apply(lambda x: x.week)
independent_variables = pd.get_dummies(independent_variables, columns=['WEEK_OF_YEAR'])

In [10]:
complete_db = daily_trx.merge(independent_variables, on =['YEAR','MONTH','YEAR_DAY'], how='left')

In [11]:
complete_db.sort_values(by=['YEAR','MONTH','YEAR_DAY'], ascending=[True,True,True], inplace=True)

* Descriptives: General

In [12]:
descriptives = pd.DataFrame()

In [13]:
descriptives = complete_db.loc[:,'TOTAL_trx'].describe().to_frame('total_trx')
descriptives = pd.concat([descriptives, complete_db.loc[:,'pn_SUM_TRX_no_t'].describe().to_frame()], axis=1, join='inner')
descriptives = pd.concat([descriptives, complete_db.loc[:,'pn_SUM_TRX_3t'].describe().to_frame()], axis=1, join='inner')
descriptives = pd.concat([descriptives, complete_db.loc[:,'pn_SUM_TRX_tm'].describe().to_frame()], axis=1, join='inner')
descriptives = pd.concat([descriptives, complete_db.loc[:,'zp_SUM_TRX'].describe().to_frame()], axis=1, join='inner')

In [14]:
descriptives

,total_trx,pn_SUM_TRX_no_t,pn_SUM_TRX_3t,pn_SUM_TRX_tm,zp_SUM_TRX
count,1.096000e+03,1.096000e+03,1096.000000,1096.000000,1096.000000
mean,2.416262e+06,1.675027e+06,342207.135036,138388.566606,260639.020073
std,8.398716e+05,5.751708e+05,109530.562299,229832.148323,164090.711388
min,4.875170e+05,3.253820e+05,80576.000000,0.000000,3202.000000
25%,1.669855e+06,1.250662e+06,263248.500000,4255.500000,41215.250000
50%,2.873835e+06,1.781616e+06,360663.500000,26267.500000,343826.500000
75%,3.084350e+06,2.177554e+06,435838.000000,150627.000000,384499.250000
max,3.364692e+06,2.472601e+06,510018.000000,861918.000000,477827.000000


In [15]:
#pd.set_option('display.float_format', '{:.3e}'.format)

In [16]:
#print(descriptives.to_latex())

* Choose of max_lags based on number of observations

In [17]:
import math

In [18]:
g_1 = math.floor(4*math.pow((1096/100),(2/9)))
#g_2 = math.floor(math.pow(1096,1/4))

In [19]:
g_1

6

* Defining a function to estimate and summarize prediction

In [20]:
def estimateWithStatsModels(Y,X,g,name):
    X = sm.add_constant(X)
    m = sm.OLS(Y, X)
    results = m.fit().get_robustcov_results(cov_type='HAC',maxlags=g)
   
    return results

### M1_1

* Complete model

In [21]:
Y = complete_db.loc[:,'TOTAL_trx']
X = complete_db.loc[:,['SATURDAY',
                       'SUNDAY',
                       'Metro Hora Punta',
                       'kms_ofertados',
                       'WEEK_OF_YEAR_52',
                       'WEEK_OF_YEAR_53',
                       'WEEK_OF_YEAR_1',
                       'WEEK_OF_YEAR_2',
                       'WEEK_OF_YEAR_3',
                       'WEEK_OF_YEAR_4',
                       'WEEK_OF_YEAR_5',
                       'WEEK_OF_YEAR_6',
                       'WEEK_OF_YEAR_7',
                       'WEEK_OF_YEAR_8',
                       'WEEK_OF_YEAR_9',
                       'Julio',
                       'Nov_2017',
                       'Dic_2017',
                       't',
                       'Feriado_laboral',
                       'Feriado_no_laboral',
                       'Elecciones',
                       'Censo',
                       'Partido',
                       'FDS_Largo',
                       'Disturbios',
                       'Corte_Metro',
                       'Retraso_Metro',
                       'Incidente_Metro',
                       'Bucle',
                       'Clima',
                       'visperas_laborales',
                       'kms_metro',
                       'N_ZPs',
                       'ratio_tm']]

results_m1_1 = estimateWithStatsModels(Y,X,g_1,'m1_1')
print(results_m1_1.summary())

                            OLS Regression Results                            
Dep. Variable:              TOTAL_trx   R-squared:                       0.978
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     7105.
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:55:03   Log-Likelihood:                -14410.
No. Observations:                1096   AIC:                         2.889e+04
Df Residuals:                    1060   BIC:                         2.907e+04
Df Model:                          35                                         
Covariance Type:                  HAC                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -2.926e+05   1.03

### M1_2

In [23]:
complete_db.loc[:,'log(TOTAL_trx)'] = complete_db['TOTAL_trx'].apply(lambda x: math.log(x))

In [24]:
Y = complete_db.loc[:,'log(TOTAL_trx)']
X = complete_db.loc[:,['SATURDAY',
                       'SUNDAY',
                       'Metro Hora Punta',
                       'kms_ofertados',
                       'WEEK_OF_YEAR_52',
                       'WEEK_OF_YEAR_53',
                       'WEEK_OF_YEAR_1',
                       'WEEK_OF_YEAR_2',
                       'WEEK_OF_YEAR_3',
                       'WEEK_OF_YEAR_4',
                       'WEEK_OF_YEAR_5',
                       'WEEK_OF_YEAR_6',
                       'WEEK_OF_YEAR_7',
                       'WEEK_OF_YEAR_8',
                       'WEEK_OF_YEAR_9',
                       'Julio',
                       'Nov_2017',
                       'Dic_2017',
                       't',
                       'Feriado_laboral',
                       'Feriado_no_laboral',
                       'Elecciones',
                       'Censo',
                       'Partido',
                       'FDS_Largo',
                       'Disturbios',
                       'Corte_Metro',
                       'Retraso_Metro',
                       'Incidente_Metro',
                       'Bucle',
                       'Clima',
                       'visperas_laborales',
                       'kms_metro',
                       'N_ZPs',
                       'ratio_tm']]

results_m1_2 = estimateWithStatsModels(Y,X,g_1,'m1_2')
print(results_m1_2.summary())

                            OLS Regression Results                            
Dep. Variable:         log(TOTAL_trx)   R-squared:                       0.976
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                 2.002e+04
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:55:15   Log-Likelihood:                 1364.0
No. Observations:                1096   AIC:                            -2656.
Df Residuals:                    1060   BIC:                            -2476.
Df Model:                          35                                         
Covariance Type:                  HAC                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 13.9369      0

In [25]:
complete_db.loc[:,'log(TOTAL_trx)']

0       13.361160
1       14.574382
2       14.188385
3       13.840231
4       14.911875
5       14.904786
6       14.896607
7       14.889432
8       14.880573
9       14.323844
10      13.864519
11      14.871322
12      14.838315
13      14.860947
14      14.865273
15      14.870994
16      14.293090
17      13.844851
18      14.839287
19      14.838884
20      14.829023
21      14.815629
22      14.821769
23      14.249993
24      13.807660
25      14.789756
26      14.789511
27      14.782777
28      14.787140
29      14.819312
          ...    
1066    14.405987
1067    13.910700
1068    14.966207
1069    14.974960
1070    14.964984
1071    14.975336
1072    14.132356
1073    14.265280
1074    13.850629
1075    14.942611
1076    14.939844
1077    14.936339
1078    14.926048
1079    14.916188
1080    14.453103
1081    14.027765
1082    14.920114
1083    14.930239
1084    14.931916
1085    14.915863
1086    14.906420
1087    14.400822
1088    13.854052
1089    13.355018
1090    14

## Closed